# SVHN: Evaluation Section

## Imports and SVHN loading

In [1]:
# Imports all the module paths
import sys

import torch

sys.path.append("../../")

# Loads the rest of the modules

# File containing all the required training methods
import defences.cifar10 as defences

# For testing
import utils.clean_test as clean_test

# Contains the data loadders
import utils.dataloaders as dataloaders

# For printing outcomes
# import utils.printing as printing

# Example printing, but I removed it to simplify results
# for epsilon in epsilons:
#     printing.print_attack(
#         model,
#         testSetLoader,
#         "FGSM",
#         attacks["FGSM"],
#         epsilon=epsilon,
#     )

Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA


## Load the dataset

In [2]:
DATA_ROOT = "../../datasets/SVHN"

trainSetLoader, _, testSetLoader = dataloaders.get_SVHN_data_loaders(
    DATA_ROOT,
    trainSetSize=63257,
    validationSetSize=0,
    batchSize=128,
)

Using downloaded and verified file: ../../datasets/SVHN/train_32x32.mat
Using downloaded and verified file: ../../datasets/SVHN/extra_32x32.mat
Using downloaded and verified file: ../../datasets/SVHN/train_32x32.mat
Using downloaded and verified file: ../../datasets/SVHN/test_32x32.mat


## Save path

In [3]:
SAVE_LOAD_ROOT = "../../models_data/SVHN"

## Load the Attacks For Training

In [4]:
# A possible attacks array (for nice printing):
# Some attacks use a helper library
import torchattacks

import attacks.fgsm as fgsm
import attacks.ifgsm as ifgsm
import attacks.pgd as pgd
import utils.attacking as attacking

attacks = {}

attacks["FGSM"] = fgsm.fgsm_attack
attacks["I-FGSM"] = ifgsm.ifgsm_attack
attacks["PGD"] = pgd.pgd_attack

## Baseline performance on standard model

In [5]:
standard_model = defences.standard_training(
    trainSetLoader,
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/svhn_standard",
    long_training=False,
)

Found already trained model...
... loaded!


In [6]:
# Test the model
clean_test.test_trained_model(standard_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 95.09%


In [7]:
# Save the model
torch.save(standard_model, SAVE_LOAD_ROOT + "/svhn_standard")

## Training Phase of Framework

In [8]:
framework_trained_model = defences.framework_training(
    trainSetLoader,
    attack_function1=attacks["PGD"],
    attack_function2=None,
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/svhn_framework",
    epsilon1=(8 / 255),
    alpha=(2 / 255),
    iterations=7,
    steps=15,
    c=0.15,
)

Found already trained model...
... loaded!


In [9]:
# Test the model
clean_test.test_trained_model(framework_trained_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 89.66%


In [10]:
# Save the model
torch.save(framework_trained_model, SAVE_LOAD_ROOT + "/svhn_framework")

In [11]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [12]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        framework_trained_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 89.66%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 45.25%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 36.03%


Testing the model under FGSM Attack using epsilon = 0.75, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 37.59%


Testing the model under FGSM Attack using epsilon = 1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 37.87%




In [13]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [14]:
for epsilon in epsilons:
    attacking.attack_model(
        framework_trained_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# deepfool_attack = torchattacks.DeepFool(framework_trained_model, steps=10)
# attacks["DeepFool"] = deepfool_attack

In [ ]:
# attacking.attack_model(
#     framework_trained_model,
#     testSetLoader,
#     "DeepFool",
#     attacks["DeepFool"],
#     library=True,
# )

In [ ]:
cw_attack = torchattacks.CW(framework_trained_model, c=1, steps=50)
attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    framework_trained_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(framework_trained_model, c=0.1, steps=50)
attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    framework_trained_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(framework_trained_model, c=0.1, steps=25)
attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    framework_trained_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(framework_trained_model, c=0.1, steps=15)
attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    framework_trained_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(framework_trained_model, c=0.05, steps=15)
attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    framework_trained_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)